In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import os
import multiprocessing as mp

# ==========================================
# Block 1: 高性能环境与数据预处理
# ==========================================

# 1. 绝对路径定义
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
file_edges = os.path.join(base_path, "edges_fully_aligned.csv")
file_od = os.path.join(base_path, "od_demand_matrix_calibrated.csv")

print("--- 开始 Block 1: 数据预处理与环境初始化 ---")

# 2. 加载核心数据集
# 加载路网数据 (56万条边)
edges_df = pd.read_csv(file_edges)
# 加载校准后的需求矩阵 (5,000对 OD)
od_df = pd.read_csv(file_od)

# 3. 路网数据清洗与容量鲁棒性处理 (数据防炸逻辑)
# BPR公式中饱和度 V/C 的分母不能为 0。确保所有 capacity 大于 0
# 对于支路或缺失数据，设定最小逻辑容量阈值 (如 100 辆/日)
min_capacity = 100
edges_df['capacity'] = edges_df['capacity'].apply(lambda x: max(x, min_capacity))

# 确保自由流时间 (T0) 为正数
edges_df['free_flow_time'] = edges_df['free_flow_time'].apply(lambda x: max(x, 0.01))

# 4. 向量化参数准备 (用于高性能计算)
# 将 BPR 需要的常数转化为 NumPy 数组，以支持 Block 2 中的矩阵运算
t0 = edges_df['free_flow_time'].values
capacity = edges_df['capacity'].values
alpha = 0.15 # BPR 标准参数
beta = 4     # BPR 标准参数

# 5. 构建 NetworkX 高性能图对象
# 预计算初始状态下的阻抗 (V=0 时的自由流时间) 作为搜索起点
G = nx.from_pandas_edgelist(
    edges_df, 'u', 'v', 
    edge_attr=['free_flow_time', 'capacity'], 
    create_using=nx.Graph()
)

# 6. 多进程并行环境初始化
# 获取系统 CPU 核心数，留出 1-2 个核心保证系统稳定
num_cores = max(1, mp.cpu_count() - 2)
print(f"检测到系统核心数: {mp.cpu_count()}, 将使用 {num_cores} 个核心进行并行计算。")

# 7. 任务分片 (将 5,000 对 OD 划分为 Batch 以优化并行效率)
od_pairs = od_df[['origin', 'destination', 'demand_qij']].values
batch_size = int(np.ceil(len(od_pairs) / num_cores))
od_batches = [od_pairs[i:i + batch_size] for i in range(0, len(od_pairs), batch_size)]

print(f"数据处理完成:")
print(f"- 已加载边数: {len(edges_df):,}")
print(f"- 已加载 OD 对: {len(od_df):,}")
print(f"- 路网平均容量: {edges_df['capacity'].mean():.2f}")
print("--- Block 1 准备就绪，可以执行 Block 2 ---")

# 输出对齐后的结果，供后续 Block 直接引用
# 核心变量: G, od_pairs, t0, capacity, alpha, beta, num_cores

--- 开始 Block 1: 数据预处理与环境初始化 ---
检测到系统核心数: 14, 将使用 12 个核心进行并行计算。
数据处理完成:
- 已加载边数: 89,655
- 已加载 OD 对: 5,000
- 路网平均容量: 30002.34
--- Block 1 准备就绪，可以执行 Block 2 ---


In [2]:
import pandas as pd
import numpy as np
import os

# ==========================================
# Block 2: BPR 引擎与 Beckmann 目标函数
# ==========================================

# 1. 路径设置 (沿用绝对路径)
base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
file_edges = os.path.join(base_path, "edges_fully_aligned.csv")
file_readme = os.path.join(base_path, "README_Block2.txt")

print("--- 开始 Block 2: BPR 引擎构建 ---")

# 2. 核心数学函数 (向量化实现)

def update_travel_time(flow, t0, capacity, alpha=0.15, beta=4):
    """
    BPR路阻函数: 计算包含拥堵延迟的实时通行时间
    T = T0 * (1 + alpha * (V/C)^beta)
    """
    # 向量化运算，flow/t0/capacity 均为 NumPy 数组
    travel_time = t0 * (1.0 + alpha * (flow / capacity)**beta)
    return travel_time

def calculate_beckmann_objective(flow, t0, capacity, alpha=0.15, beta=4):
    """
    Beckmann 目标函数: 计算全网阻抗积分之和
    Z = Sum( Va*T0 * (1 + (alpha/(beta+1)) * (Va/Ca)^beta) )
    """
    # 计算每一条边的积分值
    integrals = flow * t0 * (1.0 + (alpha / (beta + 1.0)) * (flow / capacity)**beta)
    # 返回全网总和 (标量)
    return np.sum(integrals)

# 3. 数据载入与初始化
edges_df = pd.read_csv(file_edges)

# 将核心字段转化为向量，确保计算效率
# 鲁棒性处理：确保容量 > 0
edges_df['capacity'] = edges_df['capacity'].replace(0, 100) 
t0_vec = edges_df['free_flow_time'].values
capacity_vec = edges_df['capacity'].values

# 初始化流量向量 (初始状态全网流量为 0)
current_flow = np.zeros(len(edges_df))

# 4. 示例演示 (机理验证)
initial_obj = calculate_beckmann_objective(current_flow, t0_vec, capacity_vec)
print(f"初始状态 (V=0) Beckmann 目标函数值: {initial_obj:,.4f}")

# 模拟一个 50% 饱和度的场景测试 BPR 引擎
test_flow = capacity_vec * 0.5
updated_time = update_travel_time(test_flow, t0_vec, capacity_vec)
print(f"测试场景 (V/C=0.5) 平均通行时间: {np.mean(updated_time):.2f} min")
print(f"相比自由流时间 (V=0) 增长率: {((np.mean(updated_time)/np.mean(t0_vec))-1)*100:.2f}%")

# 5. 生成说明文档
readme_content = """MCM/ICM 2025 Problem D - Block 2: BPR Engine & Beckmann Objective
----------------------------------------------------------------------
本模块实现了交通流分配的核心反馈机制：
1. update_travel_time: 基于流量 V 计算非线性延时 T。
2. calculate_beckmann_objective: 计算用于收敛判断的全局目标函数。
这些函数利用 NumPy 向量化技术，专门针对 8.9 万条边的大规模路网进行了性能优化。
"""
with open(file_readme, "w", encoding='utf-8') as f:
    f.write(readme_content)

print(f"--- Block 2 完成 ---")
print(f"说明文档已保存至: {file_readme}")

--- 开始 Block 2: BPR 引擎构建 ---
初始状态 (V=0) Beckmann 目标函数值: 0.0000
测试场景 (V/C=0.5) 平均通行时间: 15.30 min
相比自由流时间 (V=0) 增长率: 0.94%
--- Block 2 完成 ---
说明文档已保存至: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\README_Block2.txt


import pandas as pd
import numpy as np
import networkx as nx
import os
from multiprocessing import Pool, cpu_count
from functools import partial

# ==========================================
# Block 3: 并行化全有全无分配 (Parallel AON)
# ==========================================

# 1. 核心 Worker 函数: 在子进程中执行路径搜索
def worker_aon_assignment(od_chunk, edge_id_map, edges_list, weight_dict):
    """
    单个进程的任务：计算分配给它的 OD 块的最短路径并累加流量
    """
    # 局部图构建（每个子进程拥有独立的图对象以避免冲突）
    local_G = nx.Graph()
    for u, v, weight in edges_list:
        local_G.add_edge(u, v, weight=weight_dict.get((u, v), weight))
    
    local_flow = np.zeros(len(edge_id_map))
    
    for origin, destination, demand in od_chunk:
        try:
            # 执行 Dijkstra 寻找最短路径
            path = nx.shortest_path(local_G, source=int(origin), target=int(destination), weight='weight')
            
            # 将需求加载到路径上的每一条边
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1]
                # 检查边在 edge_id_map 中的索引
                edge_key = tuple(sorted((u, v))) # 处理无向图
                if edge_key in edge_id_map:
                    idx = edge_id_map[edge_key]
                    local_flow[idx] += demand
        except nx.NetworkXNoPath:
            continue
            
    return local_flow

# 2. 主执行函数
def run_parallel_aon(edges_df, od_df, current_weights, num_workers=12):
    """
    协调多进程执行全有全无分配
    """
    print(f"--- 启动并法化 AON 计算 (使用 {num_workers} 核心) ---")
    
    # 准备边映射表: (u, v) -> index
    edge_id_map = {}
    edges_list = []
    weight_dict = {}
    for i, row in edges_df.iterrows():
        u, v = int(row['u']), int(row['v'])
        key = tuple(sorted((u, v)))
        edge_id_map[key] = i
        edges_list.append((u, v, current_weights[i]))
        weight_dict[(u, v)] = current_weights[i]

    # OD 任务切分
    od_tasks = od_df[['origin', 'destination', 'demand_qij']].values
    chunks = np.array_split(od_tasks, num_workers)
    
    # 并行池启动
    # 固定核心组件，传入 partial 函数
    worker_func = partial(worker_aon_assignment, edge_id_map=edge_id_map, 
                          edges_list=edges_list, weight_dict=weight_dict)
    
    with Pool(processes=num_workers) as pool:
        results = pool.map(worker_func, chunks)
    
    # 结果聚合 (Reduction)
    Y_auxiliary_flow = np.sum(results, axis=0)
    
    return Y_auxiliary_flow

# 3. 示例测试与 snapshot 输出
if __name__ == "__main__":
    base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
    edges_file = os.path.join(base_path, "edges_fully_aligned.csv")
    od_file = os.path.join(base_path, "od_demand_matrix_calibrated.csv")
    snapshot_file = os.path.join(base_path, "AON_debug_snapshot.txt")

    # 预载入数据
    edges = pd.read_csv(edges_file)
    od = pd.read_csv(od_file)
    
    # 模拟 Iteration 0 (使用自由流时间)
    t0 = edges['free_flow_time'].values
    
    Y = run_parallel_aon(edges, od, t0, num_workers=12)
    
    # 结果审计
    total_flow_assigned = np.sum(Y)
    print(f"--- Block 3 完成 ---")
    print(f"全网总需求量: {od['demand_qij'].sum():,.2f}")
    print(f"AON 已成功分配流量: {total_flow_assigned:,.2f}")
    
    with open(snapshot_file, "w") as f:
        f.write(f"AON Snapshot\nTotal Demand: {od['demand_qij'].sum()}\nTotal Assigned: {total_flow_assigned}")

# 考虑将上述计算交给大模型云端进行 我直接让其输出数据集

import pandas as pd
import numpy as np
import networkx as nx

def generate_aon_results():
    print("正在加载数据...")
    edges_df = pd.read_csv('edges_fully_aligned.csv')
    od_df = pd.read_csv('od_demand_matrix_calibrated.csv')
    
    # 1. 构建无向图 (依据你的 original 代码逻辑)
    G = nx.Graph()
    edge_id_map = {}
    current_weights = edges_df['free_flow_time'].values
    
    for i, row in edges_df.iterrows():
        u, v = int(row['u']), int(row['v'])
        key = tuple(sorted((u, v)))
        edge_id_map[key] = i
        G.add_edge(u, v, weight=current_weights[i])
    
    # 2. 初始化流量数组
    y_flow = np.zeros(len(edges_df))
    unique_origins = od_df['origin'].unique()
    
    print(f"开始计算 AON 分配 (共 {len(unique_origins)} 个起点)...")
    
    for count, origin in enumerate(unique_origins, 1):
        origin = int(origin)
        if not G.has_node(origin): continue
            
        # 使用 Dijkstra 计算该起点到所有节点的最短路径
        paths = nx.single_source_dijkstra_path(G, origin, weight='weight')
        
        # 提取该起点的所有 OD 需求
        origin_demands = od_df[od_df['origin'] == origin]
        
        for _, row in origin_demands.iterrows():
            dest = int(row['destination'])
            demand = row['demand_qij']
            if dest in paths:
                p = paths[dest]
                for i in range(len(p) - 1):
                    edge_key = tuple(sorted((p[i], p[i+1])))
                    if edge_key in edge_id_map:
                        y_flow[edge_id_map[edge_key]] += demand
        
        if count % 20 == 0:
            print(f"已完成: {count}/{len(unique_origins)}")

    # 3. 保存结果
    edges_df['assigned_flow'] = y_flow
    edges_df.to_csv('edges_with_aon_flow_LOCAL.csv', index=False)
    
    print("\n--- 计算完成 ---")
    print(f"结果已保存至: edges_with_aon_flow_LOCAL.csv")
    print(f"总分配流量验证: {np.sum(y_flow):,.2f}")

if __name__ == "__main__":
    generate_aon_results()

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import os

def generate_aon_results_absolute():
    # ================= 配置绝对路径 =================
    base_path = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
    edges_file = os.path.join(base_path, "edges_fully_aligned.csv")
    od_file = os.path.join(base_path, "od_demand_matrix_calibrated.csv")
    output_file = os.path.join(base_path, "edges_with_aon_flow.csv")
    # ===============================================

    print(f"1. 正在从以下路径读取数据:\n   {base_path}")
    
    if not os.path.exists(edges_file) or not os.path.exists(od_file):
        print("错误：找不到指定的 CSV 文件，请检查路径是否正确。")
        return

    edges_df = pd.read_csv(edges_file)
    od_df = pd.read_csv(od_file)
    
    print(f"2. 正在构建网络图 (边数: {len(edges_df)})...")
    G = nx.Graph()
    
    # 快速构建图：提取 u, v 和权重列
    # 注意：这里使用 free_flow_time 作为初始权重
    edge_data = zip(edges_df['u'].astype(int), 
                    edges_df['v'].astype(int), 
                    edges_df['free_flow_time'])
    G.add_weighted_edges_from(edge_data)
    
    # 预先构建边键 (u, v) 到索引的映射，处理无向图逻辑
    edge_keys = [tuple(sorted((int(u), int(v)))) for u, v in zip(edges_df['u'], edges_df['v'])]
    edge_id_map = {key: i for i, key in enumerate(edge_keys)}
    
    # 3. 初始化流量数组
    y_flow = np.zeros(len(edges_df))
    
    # 4. 按起点进行分组计算
    unique_origins = od_df['origin'].unique()
    print(f"3. 开始计算 AON 流量分配 (总起点数: {len(unique_origins)})...")
    
    od_groups = od_df.groupby('origin')
    
    for count, (origin, group) in enumerate(od_groups, 1):
        origin_node = int(origin)
        if not G.has_node(origin_node):
            continue
            
        # 核心优化：一次性计算出该起点到全网所有可达节点的最短路径
        # single_source_dijkstra_path 会返回一个字典：{终点: [路径节点列表]}
        try:
            paths = nx.single_source_dijkstra_path(G, origin_node, weight='weight')
        except Exception:
            continue
        
        # 遍历该起点下的所有目的地需求
        for _, row in group.iterrows():
            dest_node = int(row['destination'])
            demand = row['demand_qij']
            
            if dest_node in paths:
                p = paths[dest_node]
                # 将流量加载到路径上的每一条边
                for i in range(len(p) - 1):
                    key = tuple(sorted((p[i], p[i+1])))
                    if key in edge_id_map:
                        idx = edge_id_map[key]
                        y_flow[idx] += demand
        
        if count % 20 == 0:
            print(f"   进度: 已完成 {count}/{len(unique_origins)} 个起点的路径分配...")

    # 5. 合并结果并保存
    print(f"4. 正在保存结果至: {output_file}")
    edges_df['assigned_flow'] = y_flow
    edges_df.to_csv(output_file, index=False)
    
    print("\n" + "="*30)
    print("计算成功完成！")
    print(f"总分配流量 (Sum of Flows): {np.sum(y_flow):,.2f}")
    print("="*30)

if __name__ == "__main__":
    generate_aon_results_absolute()

1. 正在从以下路径读取数据:
   D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data
2. 正在构建网络图 (边数: 89655)...
3. 开始计算 AON 流量分配 (总起点数: 100)...
   进度: 已完成 20/100 个起点的路径分配...
   进度: 已完成 40/100 个起点的路径分配...
   进度: 已完成 60/100 个起点的路径分配...
   进度: 已完成 80/100 个起点的路径分配...
   进度: 已完成 100/100 个起点的路径分配...
4. 正在保存结果至: D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data\edges_with_aon_flow.csv

计算成功完成！
总分配流量 (Sum of Flows): 120,465,331.26


# 2025 MCM Problem D 实战指南
## 第二阶段：Block 3 复盘与技术性能分析 (Stage 2: Block 3 Review)

在完成 **Block 3 (AON Assignment)** 任务后，我们对生成的 `edges_with_aon_flow.csv` 进行了系统性审计，并针对计算过程中的性能表现进行了深度分析。

---

## 1. 数据集检查结果 (Dataset Audit)

通过对输出文件的初步分析，确认 AON 分配阶段的计算目标已达成：

* **字段完整性**：文件中已成功增加了 `assigned_flow` 字段。
* **分配特征分布**：
    * **覆盖率**：在全网 89,655 条边中，共有 2,964 条边的流量大于 0（约占总体的 **3.3%**）。这符合 AON 分配的数学特性，即流量会严格集中在理论最短路径上，导致网络覆盖呈现高度稀疏性。
    * **流量规模**：流量最大值达到了 **1,106,741**，平均流量约为 1,343。
* **结论**：流量已成功根据最短路径逻辑注入路网，为后续 **Block 4 (UE 迭代)** 提供了必要的起始流量向量。



---

## 2. 计算效率与算法优化思考 (Algorithmic Analysis)

针对大规模图计算（8.9 万条边，5,000 对 OD）中可能出现的“计算缓慢”问题，我们从以下维度进行了技术复盘：

### A. 算法复杂度优化 (Complexity Optimization)
全有全无分配的核心是路径搜索。传统的针对每一个 OD 对调用一次 Dijkstra 算法，其计算量为 $O(N_{OD} \times E \log V)$。

* **优化策略 (SSSP)**：应采用 **单源最短路径 (Single Source Shortest Path)** 逻辑。针对每一个“起点（Origin）”，计算其到所有“终点（Destinations）”的一棵最短路径树。
    * **收益**：计算次数从“OD 对数量”减少到了“起点数量”，复杂度降至 $O(N_{Origin} \times E \log V)$。
* **数据结构优化**：NetworkX 作为纯 Python 实现，在处理大规模对象时存在内存开销。
    * **建议**：在高性能场景下，可考虑更换基于 C++/Rust 实现的底层引擎（如 `igraph` 或 `graph-tool`）。

---

## 3. CPU 调用率瓶颈分析 (Hardware Utilization)

在计算过程中观察到 CPU 调用率偏低（10%-15%），其深层原因主要源于以下物理限制：

### B. 性能瓶颈根源
1.  **Python GIL (Global Interpreter Lock)**：这是最核心的原因。标准的 Python (CPython) 执行过程中，GIL 确保同一时刻只有一个 CPU 核心在执行字节码。即便拥有多核硬件，单线程程序也无法实现真正的物理并行。

2.  **I/O 与内存阻塞**：计算过程中的频繁磁盘读写（I/O Wait）以及由于数据量庞大触发的垃圾回收 (GC) 或虚拟内存交换 (Swapping)，会导致 CPU 进入“空等”状态。
3.  **并行化缺失**：交通分配是典型的 **“尴尬并行” (Embarrassingly Parallel)** 任务。

### C. 优化路径：多进程并发
为了突破性能上限，必须引入 `multiprocessing` 模块：
* **实现逻辑**：将起点列表 (Origins) 拆分为多个分片 (Chunks)，分配给不同的进程执行。
* **预期效果**：这将瞬间打破 GIL 限制，使 CPU 调用率提升至 **100%**，极大加速 Block 4 阶段的高频率迭代过程。

---

## 4. 总结与下一步建议

目前 Block 3 已经建立了一个稳健的流量底座。但由于 **用户均衡分配 (User Equilibrium)** 需要进行多次 AON 迭代，目前的单线程效率将成为大规模仿真的瓶颈。

**下一步您可以：**
* **引入多进程并行化**：在 Block 4 的迭代引擎中部署并发搜索逻辑。
* **执行大桥倒塌仿真**：移除 `G.remove_edge()` 对应的边，并在残损网络上重新运行该分配逻辑。
* **计算系统延时**：量化灾前与灾后全网总旅行时间 ($TSTT$) 的变动量。

**您是否需要我为您提供一份关于“如何在 Python 中利用 `multiprocessing` 加速交通分配”的参考代码块？**

In [7]:
import pandas as pd
import numpy as np
import os
import time
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra
from scipy.optimize import minimize_scalar

# ==========================================
# 1. 配置与路径设定
# ==========================================
DATA_PATH = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"

ALPHA = 0.15
BETA = 4
MAX_ITER = 500  # 建议增加到 30 次以观察收敛趋势
TOLERANCE = 1e-4

# ==========================================
# 2. 核心函数
# ==========================================

def bpr_time(t0, capacity, flow):
    return t0 * (1 + ALPHA * np.power((flow / capacity), BETA))

def beckmann_obj(alpha, current_flow, target_flow, t0, capacity):
    # 计算当前混合流量：x = x_old + alpha * (y - x_old)
    x = current_flow + alpha * (target_flow - current_flow)
    # Beckmann 目标函数：各路段 BPR 积分之和
    val = np.sum(t0 * (x + (ALPHA / (BETA + 1)) * (np.power(x, BETA + 1) / np.power(capacity, BETA))))
    return val

def run_aon_step(od_grouped, n_nodes, graph_data, edge_map):
    """
    全有全无分配 (AON) 核心逻辑
    """
    row, col, weights = graph_data
    graph = csr_matrix((weights, (row, col)), shape=(n_nodes, n_nodes))
    target_flow = np.zeros(len(weights))
    
    # 针对每个唯一起点进行 SSSP
    for o_idx, group in od_grouped:
        dist, predecessors = dijkstra(graph, indices=o_idx, return_predecessors=True)
        
        d_indices = group['d_idx'].values
        demands = group['demand_qij'].values
        
        for d_idx, demand in zip(d_indices, demands):
            curr = d_idx
            # 回溯路径
            while curr != o_idx and curr != -9999:
                prev = predecessors[curr]
                if prev == -9999: break
                
                # 快速定位边索引
                if (prev, curr) in edge_map:
                    target_flow[edge_map[(prev, curr)]] += demand
                curr = prev
    return target_flow

# ==========================================
# 3. 主程序
# ==========================================

def run_ue_assignment():
    print(f"[{time.strftime('%H:%M:%S')}] 加载数据中...")
    
    edges = pd.read_csv(os.path.join(DATA_PATH, 'edges_with_aon_flow.csv'))
    od = pd.read_csv(os.path.join(DATA_PATH, 'od_demand_matrix_calibrated.csv'))

    # 节点与边映射
    all_nodes = np.unique(np.concatenate([edges['u'], edges['v'], od['origin'], od['destination']]))
    node_to_idx = {node: i for i, node in enumerate(all_nodes)}
    n_nodes = len(all_nodes)
    
    edges['u_idx'] = edges['u'].map(node_to_idx)
    edges['v_idx'] = edges['v'].map(node_to_idx)
    od['o_idx'] = od['origin'].map(node_to_idx)
    od['d_idx'] = od['destination'].map(node_to_idx)
    
    edge_map = {(r.u_idx, r.v_idx): i for i, r in edges.iterrows()}
    t0 = edges['free_flow_time'].values
    capacity = edges['capacity'].values
    od_grouped = list(od.groupby('o_idx'))

    # ------------------------------------------
    # Step 0: 初始化 (第一次 AON 分配)
    # ------------------------------------------
    print(f"[{time.strftime('%H:%M:%S')}] Step 0: 执行初始 AON 分配 (基于自由流时间)...")
    graph_data_0 = (edges['u_idx'].values, edges['v_idx'].values, t0)
    current_flow = run_aon_step(od_grouped, n_nodes, graph_data_0, edge_map)
    
    print(f"[{time.strftime('%H:%M:%S')}] 初始分配完成。开始 Frank-Wolfe 迭代...")

    # ------------------------------------------
    # 迭代计算
    # ------------------------------------------
    for i in range(MAX_ITER):
        iter_start = time.time()
        
        # 1. 更新当前阻抗
        current_weights = bpr_time(t0, capacity, current_flow)
        
        # 2. 寻找下降方向 (再次 AON)
        graph_data = (edges['u_idx'].values, edges['v_idx'].values, current_weights)
        target_flow = run_aon_step(od_grouped, n_nodes, graph_data, edge_map)
        
        # 3. 一维搜索寻找最佳步长 alpha
        # 搜索范围为 [0, 1]
        res = minimize_scalar(beckmann_obj, bounds=(0, 1), 
                              args=(current_flow, target_flow, t0, capacity), 
                              method='bounded')
        alpha = res.x
        
        # 4. 更新流量
        new_flow = current_flow + alpha * (target_flow - current_flow)
        
        # 5. 计算收敛指标 (Relative Gap)
        # Gap = sum(cost * (target - current)) / sum(cost * current)
        num = np.sum(current_weights * (target_flow - current_flow))
        den = np.sum(current_weights * current_flow)
        gap = abs(num / den) if den != 0 else 1
        
        duration = time.time() - iter_start
        print(f"迭代 {i+1:02d} | 步长: {alpha:.4f} | Gap: {gap:.6e} | 耗时: {duration:.1f}s")
        
        current_flow = new_flow
        
        if gap < TOLERANCE:
            print(f"[{time.strftime('%H:%M:%S')}] 满足收敛精度 ({TOLERANCE})。")
            break

    # 保存
    edges['ue_flow'] = current_flow
    edges['ue_travel_time'] = bpr_time(t0, capacity, current_flow)
    edges['v_c_ratio'] = current_flow / capacity
    
    output_path = os.path.join(DATA_PATH, 'edges_with_ue_flow.csv')
    try:
        edges.to_csv(output_path, index=False)
        print(f"成功保存至: {output_path}")
    except PermissionError:
        # 如果文件被 Excel 占用，自动加上时间戳保存，防止数据丢失
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        alt_path = output_path.replace(".csv", f"_{timestamp}.csv")
        edges.to_csv(alt_path, index=False)
        print(f"警告：原文件被占用，结果已另存为: {alt_path}")

if __name__ == "__main__":
    run_ue_assignment()

[15:49:10] 加载数据中...
[15:49:15] Step 0: 执行初始 AON 分配 (基于自由流时间)...
[15:49:18] 初始分配完成。开始 Frank-Wolfe 迭代...
迭代 01 | 步长: 0.3151 | Gap: 9.803021e-01 | 耗时: 3.5s
迭代 02 | 步长: 0.1191 | Gap: 4.342856e-01 | 耗时: 3.5s
迭代 03 | 步长: 0.1208 | Gap: 5.777047e-01 | 耗时: 3.4s
迭代 04 | 步长: 0.0683 | Gap: 2.824936e-01 | 耗时: 3.6s
迭代 05 | 步长: 0.0439 | Gap: 2.285990e-01 | 耗时: 3.1s
迭代 06 | 步长: 0.0307 | Gap: 1.284049e-01 | 耗时: 3.5s
迭代 07 | 步长: 0.0277 | Gap: 1.429892e-01 | 耗时: 3.2s
迭代 08 | 步长: 0.0255 | Gap: 1.066031e-01 | 耗时: 3.8s
迭代 09 | 步长: 0.0147 | Gap: 7.227595e-02 | 耗时: 3.5s
迭代 10 | 步长: 0.0167 | Gap: 6.990062e-02 | 耗时: 3.7s
迭代 11 | 步长: 0.0135 | Gap: 7.067789e-02 | 耗时: 3.5s
迭代 12 | 步长: 0.0124 | Gap: 5.270571e-02 | 耗时: 3.5s
迭代 13 | 步长: 0.0106 | Gap: 5.365216e-02 | 耗时: 1.3s
迭代 14 | 步长: 0.0110 | Gap: 4.359062e-02 | 耗时: 3.4s
迭代 15 | 步长: 0.0077 | Gap: 3.738708e-02 | 耗时: 3.9s
迭代 16 | 步长: 0.0086 | Gap: 3.575514e-02 | 耗时: 3.9s
迭代 17 | 步长: 0.0074 | Gap: 3.868335e-02 | 耗时: 3.6s
迭代 18 | 步长: 0.0080 | Gap: 3.042825e-02 | 耗时: 3.

In [8]:
import pandas as pd
import numpy as np
import os

# ==========================================
# 1. 配置与路径设定
# ==========================================
DATA_PATH = r"D:\PyCode\论文复现与改进\2025-D\2507692\论文复现与优化\2025_Problem_D_Data"
INPUT_FILE = os.path.join(DATA_PATH, 'edges_with_ue_flow.csv')

def save_with_doc(df, filename, description):
    """保存CSV并生成配套的说明文档"""
    csv_path = os.path.join(DATA_PATH, filename)
    txt_path = csv_path.replace('.csv', '.txt')
    
    # 保存CSV (使用 utf-8-sig 兼容 Excel)
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    # 保存说明文档
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(description)
    print(f"已生成: {filename} 及其说明文档")

def run_block5_analysis():
    if not os.path.exists(INPUT_FILE):
        print(f"错误: 找不到输入文件 {INPUT_FILE}")
        return

    print("正在加载 Block 4 结果数据进行深度分析...")
    df = pd.read_csv(INPUT_FILE)

    # ==========================================
    # 2. 系统宏观指标 (System Metrics)
    # ==========================================
    tstt_ue = (df['ue_flow'] * df['ue_travel_time']).sum()
    vmt_ue = (df['ue_flow'] * df['length']).sum()
    weighted_avg_vc = (df['ue_flow'] * df['v_c_ratio']).sum() / df['ue_flow'].sum() if df['ue_flow'].sum() > 0 else 0
    utilization_ue = (df['ue_flow'] > 0).sum() / len(df) * 100
    tstt_aon = (df['assigned_flow'] * df['free_flow_time']).sum() if 'assigned_flow' in df.columns else np.nan
    efficiency_loss = (tstt_ue / tstt_aon - 1) * 100 if tstt_aon > 0 else np.nan

    system_metrics = pd.DataFrame({
        'Metric_Name': ['Total_System_Travel_Time_UE', 'Ideal_System_Travel_Time_AON', 'Efficiency_Loss_Percent', 
                        'Total_Vehicle_Distance_VMT', 'Flow_Weighted_Avg_VC', 'Network_Utilization_Rate'],
        'Value': [tstt_ue, tstt_aon, efficiency_loss, vmt_ue, weighted_avg_vc, utilization_ue]
    })

    system_doc = f"""数据集名称: analysis_system_metrics.csv
描述: 该表记录了路网在用户均衡状态下的宏观表现指标。
字段说明:
- Total_System_Travel_Time_UE: 系统总旅行耗时，反映全社会总交通成本。
- Ideal_System_Travel_Time_AON: 理想状态（无拥堵）下的总耗时。
- Efficiency_Loss_Percent: 效率损失百分比。该值越高，说明桥梁倒塌后的绕行造成的社会时间成本越高。
- Total_Vehicle_Distance_VMT: 总行驶里程。
- Flow_Weighted_Avg_VC: 流量加权后的平均饱和度。比简单平均值更真实地反映驾驶员的感受。
- Network_Utilization_Rate: 路网中有流量通过的路段比例。
"""
    save_with_doc(system_metrics, 'analysis_system_metrics.csv', system_doc)

    # ==========================================
    # 3. 服务水平统计 (LOS Stats)
    # ==========================================
    def classify_los(vc):
        if vc == 0: return 'Unused'
        if vc <= 0.6: return 'LOS_A_B_C'
        elif vc <= 0.8: return 'LOS_D'
        elif vc <= 1.0: return 'LOS_E'
        else: return 'LOS_F'

    df['los_category'] = df['v_c_ratio'].apply(classify_los)
    los_stats = df.groupby('los_category').agg(
        Edge_Count=('u', 'count'),
        Total_Flow=('ue_flow', 'sum'),
        Avg_VC=('v_c_ratio', 'mean')
    ).reset_index()

    los_doc = f"""数据集名称: analysis_los_stats.csv
描述: 根据道路饱和度（V/C比）对全路网服务水平进行分级统计。
分级标准:
- LOS A/B/C (V/C <= 0.6): 交通顺畅。
- LOS D (0.6 < V/C <= 0.8): 出现轻度拥堵。
- LOS E (0.8 < V/C <= 1.0): 流量接近饱和，非常不稳定。
- LOS F (V/C > 1.0): 严重拥堵，路段吞吐能力失效。
论文用途: 建议使用饼图展示 LOS F 路段的占比，作为路网脆弱性的证据。
"""
    save_with_doc(los_stats, 'analysis_los_stats.csv', los_doc)

    # ==========================================
    # 4. 道路等级性能分析 (Highway Performance)
    # ==========================================
    highway_perf = df.groupby('highway').agg(
        Count=('u', 'count'),
        Total_Flow=('ue_flow', 'sum'),
        Avg_VC=('v_c_ratio', 'mean'),
        Max_VC=('v_c_ratio', 'max')
    ).reset_index()

    highway_doc = f"""数据集名称: analysis_highway_performance.csv
描述: 统计不同行政等级道路在交通压力下的表现。
分析重点:
- 观察 Motorway (高速) 的 Avg_VC 是否普遍超过 1.0。
- 观察 Residential (住宅区) 道路是否出现了异常高流量（避堵绕行导致）。
字段说明:
- Count: 该等级道路的总路段数。
- Total_Flow: 该等级道路承担的总交通负荷。
- Avg_VC: 该等级道路的平均拥堵程度。
"""
    save_with_doc(highway_perf, 'analysis_highway_performance.csv', highway_doc)

    # ==========================================
    # 5. 关键瓶颈清单 (Bottleneck Inventory)
    # ==========================================
    bottlenecks = df.sort_values('v_c_ratio', ascending=False).head(100)[[
        'u', 'v', 'name', 'highway', 'ue_flow', 'capacity', 'v_c_ratio', 'ue_travel_time', 'free_flow_time'
    ]].copy()
    bottlenecks['Delay_Ratio'] = bottlenecks['ue_travel_time'] / bottlenecks['free_flow_time']

    bottleneck_doc = f"""数据集名称: analysis_bottleneck_inventory.csv
描述: 全路网中最拥堵的前 100 条“交通黑点”路段清单。
字段说明:
- Delay_Ratio: 延迟倍数。例如值为 5，表示通过该路段的时间是顺畅时的 5 倍。
- ue_flow: 该瓶颈处的实际车辆负载。
- capacity: 设计通行能力。
论文用途: 可以直接点名分析这些路段（如特定的大桥出口、主干道交汇点），作为案例研究(Case Study)。
"""
    save_with_doc(bottlenecks, 'analysis_bottleneck_inventory.csv', bottleneck_doc)

    print("\n--- 所有 Block 5 分析数据集及说明文档已生成完毕 ---")

if __name__ == "__main__":
    run_block5_analysis()

正在加载 Block 4 结果数据进行深度分析...
已生成: analysis_system_metrics.csv 及其说明文档
已生成: analysis_los_stats.csv 及其说明文档
已生成: analysis_highway_performance.csv 及其说明文档
已生成: analysis_bottleneck_inventory.csv 及其说明文档

--- 所有 Block 5 分析数据集及说明文档已生成完毕 ---
